# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-04-02 00:00:00  to  2020-04-03 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,105,100,-5,-4.761905
1,02,186,210,24,12.903226
2,03,36,40,4,11.111111
3,04,25,29,4,16.000000
4,05,59,64,5,8.474576


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,2838,2897,59,2.078929
92,92,1748,1848,100,5.720824
94,94,1474,1613,139,9.430122
93,93,1208,1267,59,4.884106
69,69,1138,1202,64,5.623902


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -13  Mean:  12.145833333333334  Max:  139


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -8.21917808219178  Mean %:  5.879661288519677  Max %:  46.15384615384615


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
0,01,105,100,-5,-4.761905
8,09,17,16,-1,-5.882353
10,11,101,99,-2,-1.980198
15,16,19,18,-1,-5.263158
23,25,186,173,-13,-6.989247
28,2A,74,72,-2,-2.702703
30,30,74,70,-4,-5.405405
36,36,49,47,-2,-4.081633
41,41,73,67,-6,-8.219178
47,47,30,29,-1,-3.333333


#### <br> Departments with highest evolution of hospitalizations (total number)

In [11]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
94,94,1474,1613,139,9.430122
92,92,1748,1848,100,5.720824
12,13,946,1022,76,8.033827
77,77,619,694,75,12.116317
91,91,758,831,73,9.630607


#### <br> Departments with highest rate of hospitalizations (%)

In [12]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
46,46,13,19,6,46.153846
58,58,15,21,6,40.000000
29,2B,12,16,4,33.333333
22,24,15,20,5,33.333333
52,52,66,82,16,24.242424


#### <br> Focus on some departments

In [13]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,2838,2897,59,2.078929


In [14]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,169,163,-6,-3.550296


In [15]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,214,226,12,5.607477
